# Zufallsbewegungen

Siehe Wiedemann/Ingold: *Numerische Physik mit Python*, Springer-Spektrum 2024, ISBN 978-3-662-69566-1

---

In diesem Jupyter-Notebook werden zufällige Bewegungen in diskreten Zeitschritten in einer Raumdimension betrachtet. Im ersten Teil ist die Schrittweite konstant, aber das Teilchen kann sich mit gleicher Wahrscheinlichkeit entweder nach links oder nach rechts bewegen. Im zweiten Teil wird eine normalverteilte Schrittweite betrachtet. Aufgrund des Zufallscharakters werden ganze Ensembles von Teilchen behandelt und Mittelwerte von relevanten Größen berechnet.

### Importanweisungen

Neben bereits bekannten Importanweisungen wird für die Achseneinteilung bei einem Balkendiagramm der Namensraum des `ticker`-Moduls von `matplotlib` importiert.

In [ ]:
from math import pi, sqrt
import numpy as np
from scipy import special
import ipywidgets as widgets
from ipywidgets import interact
import matplotlib.pyplot as plt
from matplotlib import ticker

plt.style.use("numphyspy.style")

## Zufallsbewegung mit fester Schrittweite

### Zeitentwicklung des Ensembles

Zunächst wird ein zweidimensionales Array erzeugt, das zufällig verteilt die Werte -1 und +1 enthält, die angeben, ob die Bewegung nach links oder nach rechts erfolgt. Die Achse 0 des Arrays entspricht den Zeitschritten, während die Achse 1 zu den verschiedenen Realisierungen des Ensembles gehört. Die zeitliche Bewegung erhält man durch eine kumulative Summe über die Achse 0, deren Ergebnisse im Array `trajectories` gespeichert wird. Bei der Initialisierung wird eine zusätzliche Zeile zu Beginn hinzugefügt, die den Anfangsort auf null setzt. Durch Mittelung über die einzelnen Realisierungen, also entlang der Achse 1, werden mit Hilfe der NumPy-Funktionen `mean` und `var` sowohl der Mittelwert des Orts als auch die zugehörige Varianz bestimmt.

In [ ]:
def time_development_choice(n_steps, n_ensemble):
    rng = np.random.default_rng()
    steps = rng.choice((-1, 1), size=(n_steps, n_ensemble))
    trajectories = np.zeros((n_steps+1, n_ensemble))
    trajectories[1:, :] = np.cumsum(steps, axis=0)
    x_mean = np.mean(trajectories, axis=1)
    x_var = np.var(trajectories, axis=1)
    return trajectories[-1, :], x_mean, x_var

### Berechnung der Normalverteilung

Die Funktion `gaussian` berechnet eine bei null zentrierte Normalverteilung mit Varianz `mu_2`.

In [ ]:
def gaussian(x, mu_2):
    return np.exp(-0.5*x**2/mu_2) / sqrt(2*pi*mu_2)

### Berechnung der exakten Lösung der Mastergleichung

In der Funktion `solution_master` wird die exakte Lösung der Mastergleichung nach `n_steps` Schritten berechnet. Das Ergebnis ist ein Array, dessen erster Wert zum Ort `-n_steps` und dessen letzter Wert zum Ort `n_steps` gehört.

In [ ]:
def solution_master(n_steps):
    p = np.zeros(2*n_steps+1)
    j = np.arange(-n_steps, n_steps+1, 2)
    p[::2] = special.binom(n_steps, (n_steps+j)//2
                           ) / 2**n_steps
    return p

### Implementierung der Bedienelemente und graphische Darstellung der Verteilung

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_steps_1`: Zahl der Schritte bis zur darzustellenden Verteilung
- `log_n_ensemble_1`: Zehnerlogarithmus der Ensemblegröße

Die Verteilung nach der vorgegebenen Anzahl von Schritten wird als Balkendiagramm dargestellt. Zur Berechnung der Verteilung wird dabei die NumPy-Funktion `unique` verwendet. Zum Vergleich wird die exakte Lösung der Mastergleichung durch schwarze Punkte angedeutet. Die rote Linie stellt eine Normalverteilung mit einer Varianz dar, die durch die Anzahl der Zeitschritte gegeben ist.

In [ ]:
widget_dict = {"n_steps_1":
               widgets.IntSlider(
                   value=10, min=1, max=20, step=1,
                   description=r"$n_\text{steps}$"),
               "log_n_ensemble_1":
               widgets.IntSlider(
                   value=2, min=1, max=5, step=1,
                   description=r"$\log_{10}(N)$")
               }

@interact(**widget_dict)
def plot_result_choice_1(n_steps_1, log_n_ensemble_1):
    n_ensemble_1 = 10**log_n_ensemble_1
    p, x_mean, x_var = time_development_choice(n_steps_1,
                                               n_ensemble_1)

    x_max = int(3*sqrt(n_steps_1))
    unique_values, counts = np.unique(p, return_counts=True)

    fig, ax = plt.subplots()
    ax.bar(unique_values, counts/n_ensemble_1, 1)
    ax.xaxis.set_major_locator(
        ticker.MaxNLocator(integer=True))
    x = np.linspace(-x_max, x_max, 100)
    ax.plot(x, 2*gaussian(x, n_steps_1), color="red",
            label="Normalverteilung")
    x = np.arange(-n_steps_1, n_steps_1+1)
    p_exact = solution_master(n_steps_1)
    ax.plot(x, p_exact, color="black", linestyle="None",
            marker="o", label="exakte Lösung")
    ax.set_xlim([-x_max, x_max])
    ax.set_xlabel("$j$")
    ax.set_ylabel("$P_j$")
    ax.legend(loc="upper left", fontsize="small")

### Implementierung der Bedienelemente und graphische Darstellung der Zeitentwicklung von Mittelwert und Varianz

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_steps_2`: Zahl der Schritte bis zur darzustellenden Verteilung
- `log_n_ensemble_2`: Zehnerlogarithmus der Ensemblegröße

Die Variablen werden hier anders benannt als in der vorigen Zelle, um Kopplungen zwischen den Parametern zu vermeiden.

Graphisch dargestellt werden die Entwicklung des Mittelwerts sowie der Abweichung der Varianz vom erwarteten linearen Verhalten.

In [ ]:
widget_dict = {"n_steps_2":
               widgets.IntSlider(
                   value=100, min=100, max=1000, step=100,
                   description=r"$n_\text{steps}$"),
               "log_n_ensemble_2":
               widgets.IntSlider(
                   value=3, min=1, max=5, step=1,
                   description=r"$\log_{10}(N)$")
               }

@interact(**widget_dict)
def plot_result_choice_2(n_steps_2, log_n_ensemble_2):
    n_ensemble_2 = 10**log_n_ensemble_2
    p, x_mean, x_var = time_development_choice(n_steps_2,
                                               n_ensemble_2)
    n = np.arange(n_steps_2+1)

    fig, (ax0, ax1) = plt.subplots(1, 2)
    ax0.plot(n, x_mean)
    ax0.set_xlabel("$n$")
    ax0.set_ylabel(r"$\mu_1$")
    ax1.plot(n, x_var-n)
    ax1.set_xlabel("$n$")
    ax1.set_ylabel(r"$\mu_2-n$")

## Random-Walk mit normalverteilter Schrittweite

Nun lassen wir eine variable Schrittweite zu, die einer Normalverteilung gehorcht. Die Standardabweichung der Verteilung ist gleich 1.

### Zeitentwicklung des Ensembles

Der Code entspricht im Wesentlichen dem Vorgehen in der obigen Funktion `time_development_choice`, wobei hier jedoch Zufallszahlen aus einer Normalverteilung gezogen werden, die bei null zentriert ist und Standardabweichung 1 besitzt. Zudem wird beim Initialisieren des Zufallszahlengenerators ein `seed` angegeben, der die Ergebnisse trotz des Zufallscharakters reproduzierbar macht.

In [ ]:
def time_development_normal(n_steps, n_ensemble):
    rng = np.random.default_rng(12345)
    steps = rng.normal(size=(n_steps, n_ensemble))
    trajectories = np.zeros((n_steps+1, n_ensemble))
    trajectories[1:, :] = np.cumsum(steps, axis=0)
    x_mean = np.mean(trajectories, axis=1)
    x_var = np.var(trajectories, axis=1)
    return trajectories[-1, :], x_mean, x_var

### Implementierung der Bedienelemente und graphische Darstellung der Ergebnisse

Mit Hilfe der Schieberegler lassen sich die folgenden Parameter einstellen:
- `n_steps_3`: Zahl der Schritte bis zur darzustellenden Verteilung
- `log_n_ensemble_3`: Zehnerlogarithmus der Ensemblegröße

Die graphische Darstellung umfasst die Funktionen $\langle x(n) \rangle$ und $\langle x^2(n) \rangle-n$ sowie ein Histogramm der Verteilung am Ende der Zeitentwicklung. Zum Vergleich ist eine Normalverteilung mit der zu erwartenden Varianz eingezeichnet.

In [ ]:
widget_dict = {"n_steps_3":
               widgets.IntSlider(
                   value=100, min=100, max=1000, step=100,
                   description=r"$n_\text{steps}$"),
               "log_n_ensemble_3":
               widgets.IntSlider(
                   value=3, min=1, max=5, step=1,
                   description=r"$\log_{10}(N)$")
               }

@interact(**widget_dict)
def plot_result_normal(n_steps_3, log_n_ensemble_3):
    n_ensemble_3 = 10**log_n_ensemble_3
    n = np.arange(n_steps_3+1)
    p, x_mean, x_var = time_development_normal(n_steps_3,
                                               n_ensemble_3)

    fig, axd = plt.subplot_mosaic(
        [["mu1", "mu2"], ["P", "P"]])
    axd["mu1"].plot(n, x_mean)
    axd["mu1"].set_xlabel("$n$")
    axd["mu1"].set_ylabel(r"$\mu_1$")

    axd["mu2"].plot(n, x_var-n)
    axd["mu2"].set_xlabel("$n$")
    axd["mu2"].set_ylabel(r"$\mu_2-n$")

    x_max = int(3*sqrt(n_steps_3))
    bins = np.arange(-x_max-1, x_max+1) + 0.5
    axd["P"].hist(p, bins=bins, label="Histogramm",
                  density=True)
    axd["P"].plot(bins, gaussian(bins, n_steps_3),
                  label="Normalverteilung")
    axd["P"].set_xlim([-x_max, x_max])
    axd["P"].set_xlabel("$x$")
    axd["P"].set_ylabel("$P$")
    axd["P"].legend(loc="upper left", fontsize="small")